In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
csv_path1 = Path("Resources/schools_complete[1].csv")

In [3]:
csv_path2 = Path("Resources/students_complete[1].csv")

In [4]:
school_data_to_load = csv_path1
student_data_to_load = csv_path2

school_data = pd.read_csv(school_data_to_load,encoding="utf-8")
student_data = pd.read_csv(student_data_to_load,encoding="utf-8")

In [5]:
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## scores_by_school_spending

In [6]:
school_summary = school_data_complete.groupby(['school_name'])
school_names = school_data_complete.school_name.sort_values().unique()
school_types = school_data.sort_values(by="school_name").type
school_total_students = list(school_summary.student_name.count())
school_budget = list(school_summary.budget.mean())
school_per_student_budget = [i/j for i,j in zip(school_budget,school_total_students)]
school_avg_math_score = list(school_summary.math_score.mean())
school_avg_reading_score = list(school_summary.reading_score.mean())

school_summary = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])
school_pct_passing_math = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),school_total_students)]
school_summary = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name'])
school_pct_passing_reading = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),school_total_students)]
school_overall_passing = [(i+j)/2 for i,j in zip(school_pct_passing_math,school_pct_passing_reading)]

school_summary_df = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":school_total_students,
                                  "Total School Budget":school_budget,
                                  "Per Student Budget":school_per_student_budget,
                                  "Average Math Score":school_avg_math_score,
                                  "Average Reading Score":school_avg_reading_score,
                                  "% Passing Math":school_pct_passing_math,
                                  "% Passing Reading":school_pct_passing_reading,
                                  "Overall Passing Rate":school_overall_passing})

school_summary_df = school_summary_df.reset_index(drop=True)
school_summary_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [7]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [10]:
school_spending_df=school_summary_df.copy() 

In [11]:
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_per_student_budget,bins=spending_bins,labels=labels)
school_spending_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$585-630
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
2,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$630-645
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$630-645
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$585-630
5,Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-680
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-680
8,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-680
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,$585-630


In [13]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Overall Passing Rate"].mean()

In [14]:
scores_by_spending = pd.DataFrame({
    "Average Math Score":spending_math_scores,
    "Average Reading Score":spending_reading_scores,
    "% Passing Math":spending_passing_math,
    "% Passing Reading":spending_passing_reading,
    "Overall Passing Rate":overall_passing_spending,
})
scores_by_spending 

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-630,81.899826,83.155286,87.133538,92.718205,89.925871
$630-645,78.518855,81.624473,73.484209,84.391793,78.938001
$645-680,76.997210,81.027843,66.164813,81.133951,73.649382


## scores_by_school_size

In [16]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [17]:
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"],bins=size_bins,labels=labels)
school_summary_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,Large (2000-5000)
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,Medium (1000-2000)
2,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,Large (2000-5000)
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,Large (2000-5000)
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,Medium (1000-2000)
5,Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,Large (2000-5000)
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,Small (<1000)
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,Large (2000-5000)
8,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,Large (2000-5000)
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,Small (<1000)


In [18]:
spending_math_scores = school_summary_df.groupby(["School Size"])["Average Math Score"].mean()
spending_reading_scores = school_summary_df.groupby(["School Size"])["Average Reading Score"].mean()
spending_passing_math = school_summary_df.groupby(["School Size"])["% Passing Math"].mean()
spending_passing_reading = school_summary_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_spending = school_summary_df.groupby(["School Size"])["Overall Passing Rate"].mean()

In [19]:
scores_by_size = pd.DataFrame({
    "Average Math Score":spending_math_scores,
    "Average Reading Score":spending_reading_scores,
    "% Passing Math":spending_passing_math,
    "% Passing Reading":spending_passing_reading,
    "Overall Passing Rate":overall_passing_spending,
})
scores_by_size 

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## scores_by_school_type

In [20]:
spending_math_scores = school_summary_df.groupby(["School Type"])["Average Math Score"].mean()
spending_reading_scores = school_summary_df.groupby(["School Type"])["Average Reading Score"].mean()
spending_passing_math = school_summary_df.groupby(["School Type"])["% Passing Math"].mean()
spending_passing_reading = school_summary_df.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_spending = school_summary_df.groupby(["School Type"])["Overall Passing Rate"].mean()

In [21]:
scores_by_type = pd.DataFrame({
    "Average Math Score":spending_math_scores,
    "Average Reading Score":spending_reading_scores,
    "% Passing Math":spending_passing_math,
    "% Passing Reading":spending_passing_reading,
    "Overall Passing Rate":overall_passing_spending,
})
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
